* 데이터셋을 읽어내기

 * Sprint 1 : EDA, Feature Engineering, Data Manipulation, Data Visualization
 * Sprint 2 : Hypothesis Test, CI, Bayesian
 * Sprint 3 : DR, PCA, Cluster

* 반드시 들어가야 하는 내용

 * 지역에 따라서 선호하는 게임 장르가 다를까?
 * 연도별 게임의 트랜드가 있을까?
 * 출고량이 높은 게임에 대한 분석 및 시각화 프로세스

* 발표 촬영

 * 8분 시간 엄수
 * 촬영 준비물 : 발표 슬라이드, 발표 스크립트
 * 줌을 통해 촬영(발표자의 얼굴이 나와야 함)
 * 모든 코드를 설명할 필요는 없음
 * 청중이 데이터 직군에 관해서 아무 것도 모르는 사람이라 가정

* 피드백 및 제출

 * 4.21 18:00까지 제출(zip 파일)
 * 아카이브 > Form > 프로젝트 발표 영상 제출
 * Github Repository Link > colab 파일 제출(개인 repository 생성(public) - UR CALSS에서 확인)
 * Section 1 프로젝트 피드백 그룹 - 메일을 기다려

# 로드맵

* 본 자료는 게임마다 Platform, Genre, Year, Sales라는 column을 지닌 자료

* Platform vs Sales
 * 가장 잘나가는 Platform은 무엇인가?
 * Platform 기준 지역별 점유율은 어떠한가?
 * 각 Platform 의 지역당 판매 비율은 어떠한가?

* Genre vs Sales
 * 가장 잘 팔리는 Genre는 무엇인가?
 * 지역별 선호 Genre는 어떠한가?
 * 각 Genre 의 주 판매처는 어디인가?

* Year vs Sales
 * 시장의 성장정도는 어떠한가?
 * 어느 지역의 성장도가 가장 강하고 어느 지역이 약한가?

* Platform + Genre vs Sales
 * 특정 Genre에서 유리해보이는 Platform이 있는가?

* Platform + Year vs Sales
 * Platform의 전체 성장세와 지역별 성장도(미래 비전)

* Genre + Year vs Sales
 * 특정 시기에 잘팔리는 Genre가 있나? 최근의 추세는?
 * 가장 잘 팔리는 Genre는 상승세인가? 하락세인가?
 * 가장 상승세에 있는 Genre는 무엇인가?

* Platform + Genre + Year vs Sales
 * 특정 Genre에서 상승세, 하락세를 보이는 Platform이 있나?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
url = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRBUgfYBUq8heoHl7sFbhfvRTPM58qTyzsEEJkeILT11GIFgqSt4piO_F4rvWs8w-eZjjQvXw3-2Hzz/pub?gid=1854992701&single=true&output=csv')
df = pd.DataFrame(url).drop(['Unnamed: 0'],axis=1).dropna()

In [ ]:
Sales = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Total_Sales']

In [ ]:
def to_int(x):
    return int(x)

def modify(x):
    if x < 50:
      x = x + 2000
    elif x < 100:
      x = x + 1900
    return x

df['Year'] = df['Year'].apply(to_int).apply(modify)

def to_float(x):
    x = x.replace('K','000')
    x = x.replace('M','000000')
    return float(x)

for Column_Name in Sales[:-1]:
    df[Column_Name] = df[Column_Name].apply(to_float)

df['Total_Sales'] = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales']

df = df.sort_values(by=['Year'], ascending=[False]).reset_index()
df = df.drop(['index'], axis = 1)

df

In [ ]:
Genre = []
for x in df['Genre']:
  if x not in Genre:
    Genre.append(x)
Genre.remove('Misc') # 범례를 알 수 없는경우는 뺀다

Genre

In [ ]:
# Year, Genre, Sales 만 뽑아내기 1995~2015년까지
df_ygs = pd.DataFrame(columns = ['Year','Genre']+Sales)

Year_S = 2016
Year_E = 1995

Y = Year_S
n = 0
while Y >= Year_E:
  for N1 in Genre:
    df_ygs.loc[n,'Year'] = Y
    df_ygs.loc[n,'Genre'] = N1
    n = n+1
  Y = Y - 1

Y = Year_S
while Y >= Year_E:
  for N in Genre:
    for S in Sales:
      Sum = df[df['Year']==Y][df['Genre']==N][S].sum()
      Index = df_ygs[df_ygs['Year']==Y][df_ygs['Genre']==N].index[0]
      df_ygs.loc[Index,S] = Sum
  Y = Y - 1

df_ygs

In [ ]:
plt.figure(figsize=(28,6))
S = Sales[0]

for N in Genre:
    plt.plot(df_ygs[df_ygs['Genre'] == N]['Year'],df_ygs[df_ygs['Genre'] == N][S],label=N)
    plt.legend(fontsize=14);
    plt.title(S)#+"("+str(total_share[n].round(1))+"%)")

plt.show();

In [ ]:
# Year, Genre, Sales 만 뽑아내기

Y=2022
df_YGS = pd.DataFrame(columns = ['Year','Genre']+Sales)
while Y > 1900:
  List=[]
  for x in df[df['Year'] == Y]['Genre']:
    if x not in List:
      List.append(x)
  ext = pd.DataFrame(columns = ['Year','Genre']+Sales)

  n = 0
  while n < len(List) - 1:
    dummy = df[df['Year'] == Y][df[df['Year'] == Y]['Genre'] == List[n]]
    ext.loc[n,'Year'] = Y
    ext.loc[n,'Genre'] = List[n]
    for x in Sales:
      ext.loc[n,x] = dummy[x].sum()
    n = n + 1
  df_YGS = pd.concat([df_YGS, ext])
  Y = Y - 1

df_YGS = df_YGS.reset_index()
df_YGS = df_YGS.drop(['index'], axis = 1)

df_YGS

In [ ]:
df2 = pd.DataFrame(columns = ['Year','Genre','Total_Sales'])
df2['Year'] = df_YGS['Year']
df2['Genre'] = df_YGS['Genre']
df2['Total_Sales'] = df_YGS['Total_Sales']

In [ ]:
X = df2[df2['Genre'] == 'Role-Playing']['Year']
Y1 = df2[df2['Genre'] == 'Role-Playing']['Total_Sales']
X = df2[df2['Genre'] == 'Role-Playing']['Year']
Y2 = df2[df2['Genre'] == 'Role-Playing']['Total_Sales']
plt.plot(X, Y1)

In [ ]:
def share_rate(DataFrame):
  dummy = DataFrame.copy()
  for Area in Sales:
    C_sum = dummy[Area].sum()
    dummy[Area] = 100*(dummy[Area]/C_sum)
  return dummy.round(1)

In [ ]:
def pie_graph(DataFrame,Label):
  total_share = 100 * DataFrame[Sales].sum() / DataFrame[Sales].sum()[len(Sales)-1]
  plt.figure(figsize=(32,6)) # 캔버스 크기 선택
  n = 0
  while n < len(Sales):
    Area = Sales[n]
    dummy = DataFrame.sort_values(by=[Area], ascending=[False]).copy() # Area에 따라 내림차순

    fig = plt.subplot(1,len(Sales),n+1) # 1행 len(Sales)열의 n+1 번째 캔버스가 fig
    fig.pie(x=dummy[Area],labels=dummy[Label],autopct=lambda p : '{:.2f}%'.format(p)) # fig에 파이 차트 저장? 입력?
    plt.title(Area+"("+str(total_share[n].round(1))+"%)") # fig의 title은 Area 변수

    n = n+1

  plt.show()

In [ ]:
def vs_Sales(DataFrame,Column_Name):
  List = []
  for x in DataFrame[Column_Name]:
    if x not in List:
      List.append(x)
  extract = pd.DataFrame(columns = [Column_Name]+Sales)
  n = 0
  while n < len(List) - 1:
    dummy = DataFrame[DataFrame[Column_Name] == List[n]]
    extract.loc[n,Column_Name] = List[n]
    for x in Sales:
      extract.loc[n,x] = dummy[x].sum()
    n = n+1
  return extract

In [ ]:
df_YGS_ratio = df_YGS
Y = 2022
while Y > 1900:
  for Area in Sales:
    Sum = df_YGS_ratio[Area][df_YGS_ratio['Year'] == Y].sum()
    df_YGS_ratio[Area][df_YGS_ratio['Year'] == Y] = 100*df_YGS_ratio[Area][df_YGS_ratio['Year'] == Y]/Sum
  Y = Y-1

df_YGS_ratio

In [ ]:
List = []
for x in DataFrame[Column_Name]:
  if x not in List:
    List.append(x)
extract = pd.DataFrame(columns = [Column_Name]+Sales)


n = 0
while n < len(List) - 1:
  dummy = DataFrame[DataFrame[Column_Name] == List[n]]
  extract.loc[n,Column_Name] = List[n]
  for x in Sales:
    extract.loc[n,x] = dummy[x].sum()
  n = n+1

extract

In [ ]:
# 내림차순 정리 >   .sort_values(by=['Total_Sales'], ascending=[False])

PvsS = vs_Sales(df,'Platform')

In [ ]:
pie_graph(PvsS,'Platform')

In [ ]:
GvsS = vs_Sales(df,'Genre')

In [ ]:
pie_graph(GvsS,'Genre')

In [ ]:
YvsS = vs_Sales(df,'Year')

In [ ]:
pie_graph(YvsS,'Year')

In [ ]:
def Line_graph(DataFrame,Label):
  total_share = 100 * DataFrame[Sales].sum() / DataFrame[Sales].sum()[len(Sales)-1]
  plt.figure(figsize=(32,6)) # 캔버스 크기 선택
  n = 0
  while n < len(Sales):
    dummy = DataFrame.sort_values(by=[Sales[n]], ascending=[False]).copy() # Area에 따라 내림차순
    fig = plt.subplot(1,len(Sales),n+1) # 1행 len(Sales)열의 n+1 번째 캔버스가 fig
    fig.plot(dummy[Label],dummy[Sales[n]]) # fig에 파이 차트 저장? 입력?
    plt.title(Sales[n] +"("+str(total_share[n].round(1))+"%)") # fig의 title은 Area 변수

    n = n+1

  plt.show()

In [ ]:
Line_graph(YvsS,'Year')

In [ ]:
DataFrame = YvsS
dummy = DataFrame.copy()
Label = 'Year'
total_share = 100 * DataFrame[Sales].sum() / DataFrame[Sales].sum()[len(Sales)-1]

plt.figure(figsize=(32,6)) # 캔버스 크기 선택

n = 0
fig = plt.subplot(1,len(Sales),n+1) # 1행 len(Sales)열의 n+1 번째 캔버스가 fig
plt.title(Sales[n]+"("+str(total_share[n].round(1))+"%)") # fig의 title은 Area 변수
fig.plot(dummy[Label],dummy[Sales[n]])

n = n+1
fig = plt.subplot(1,len(Sales),n+1) # 1행 len(Sales)열의 n+1 번째 캔버스가 fig
plt.title(Sales[n]+"("+str(total_share[n].round(1))+"%)") # fig의 title은 Area 변수
fig.plot(dummy[Label],dummy[Sales[n]]) # fig에 파이 차트 저장? 입력?

n = n+1
fig = plt.subplot(1,len(Sales),n+1) # 1행 len(Sales)열의 n+1 번째 캔버스가 fig
plt.title(Sales[n]+"("+str(total_share[n].round(1))+"%)") # fig의 title은 Area 변수
fig.plot(dummy[Label],dummy[Sales[n]]) # fig에 파이 차트 저장? 입력?

n = n+1
fig = plt.subplot(1,len(Sales),n+1) # 1행 len(Sales)열의 n+1 번째 캔버스가 fig
plt.title(Sales[n]+"("+str(total_share[n].round(1))+"%)") # fig의 title은 Area 변수
fig.plot(dummy[Label],dummy[Sales[n]]) # fig에 파이 차트 저장? 입력?

n = n+1
fig = plt.subplot(1,len(Sales),n+1) # 1행 len(Sales)열의 n+1 번째 캔버스가 fig
plt.title(Sales[n]+"("+str(total_share[n].round(1))+"%)") # fig의 title은 Area 변수
fig.plot(dummy[Label],dummy[Sales[n]]) # fig에 파이 차트 저장? 입력?

In [ ]:
DataFrame = YvsS
dummy = DataFrame.copy()
Label = 'Year'
total_share = 100 * DataFrame[Sales].sum() / DataFrame[Sales].sum()[len(Sales)-1]
n=0

plt.figure(figsize=(32,8)) # 캔버스 크기 선택
plt.plot(dummy[Label],dummy[Sales[n]])

plt.show()

In [ ]:
def pie_graph(DataFrame,Label):
  total_share = 100 * DataFrame[Sales].sum() / DataFrame[Sales].sum()[len(Sales)-1]
  plt.figure(figsize=(32,6)) # 캔버스 크기 선택
  n = 0
  while n < len(Sales):
    Area = Sales[n]
    dummy = DataFrame.sort_values(by=[Area], ascending=[False]).copy() # Area에 따라 내림차순

    fig = plt.subplot(1,len(Sales),n+1) # 1행 len(Sales)열의 n+1 번째 캔버스가 fig
    fig.pie(x=dummy[Area],labels=dummy[Label],autopct=lambda p : '{:.2f}%'.format(p)) # fig에 파이 차트 저장? 입력?
    plt.title(Area+"("+str(total_share[n].round(1))+"%)") # fig의 title은 Area 변수

    n = n+1

  plt.show()

In [ ]:
# 지역별 Platform 점유율은 어떤가?

plt.figure(figsize=(32,6)) # 캔버스 크기 선택

n = 0
while n < len(Sales):
  Area = Sales[n]
  dummy = PvsS.sort_values(by=[Area], ascending=[False]) # Area에 따라 내림차순

  fig = plt.subplot(1,len(Sales),n+1) # 1행 len(Sales)열의 n+1 번째 캔버스가 fig
  fig.pie(x=dummy[Area],labels=dummy['Platform'],autopct=lambda p : '{:.2f}%'.format(p)) # fig에 파이 차트 저장? 입력?
  plt.title(Area) # fig의 title은 Area 변수

  n = n+1

plt.show()

In [ ]:
#for Area in Sales:

Area = 'NA_Sales'


dummy = PvsS.sort_values(by=[Area], ascending=[False])

fig = plt.(1,figsize=(8,8)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 하얀색
ax = fig.add_subplot() ## 프래임 생성
ax.pie(x=dummy[Area],labels=dummy['Platform'],autopct=lambda p : '{:.2f}%'.format(p)) ## 파이 차트 출력

n = n+1

plt.show()

In [ ]:
# Platform 기준 지역별 점유율은 어떠한가?
PvsS_T = PvsS.sort_values(by=['NA_Sales'], ascending=[False]) ## 데이터 내림차순 정리

fig = plt.figure(figsize=(8,8)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 하얀색
 
ax = fig.add_subplot() ## 프래임 생성
 
ax.pie(x=PvsS_T['NA_Sales'],labels=PvsS_T['Platform'],autopct=lambda p : '{:.2f}%'.format(p)) ## 파이 차트 출력
plt.show()

In [ ]:
# 각 Platform 의 지역당 판매 비율은 어떠한가?